# 99 · Integración con Udacity Simulator

Este notebook orquesta el servidor `tools/sim_drive.py`:

1. Elige qué `run_dir` usar (manual o desde `best_for_sim.json`).
2. Busca el checkpoint en `task_2_circuito2`.
3. Carga el preset y valida que el modelo es compatible.
4. Lanza el servidor de inferencia (`sim_drive.py`) en un puerto concreto.
5. Tú arrancas el simulador Udacity en **Autonomous Mode** y el coche
   se conecta al servidor.

> Cuando termines, puedes parar el servidor desde aquí (matando el proceso).

In [1]:
# %%bash
# (Opcional) instala dependencias
# pip install python-socketio eventlet flask pillow opencv-python

In [ ]:
from pathlib import Path
import sys, json, subprocess, shlex, time
from datetime import datetime

# === Raíz del proyecto =======================================================
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.config import load_preset
from src.models import build_model
from src.datasets import ImageTransform

# === Override manual opcional ================================================
# Si RUN_DIR_MANUAL es None -> se usará best_for_sim.json (comportamiento viejo)
# Si pones aquí el nombre de una carpeta de outputs/ -> se ignora best_for_sim.json
#
# EJEMPLOS de nombres válidos:
#   "continual_std_sa-snn_best_sa_k013_vt120_rate_model-PilotNetSNN_66x200_gray_seed_42"
#   "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_best_as_gr050_lam020_ema090_rate_model-PilotNetSNN_66x200_gray_seed_42"
#   "continual_std_sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0_best_sca_bins50_beta050_bias0_temp050_ab24_rate_model-PilotNetSNN_66x200_gray_seed_42"
# RUN_DIR_MANUAL = "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_best_as_gr050_lam020_ema090_rate_model-PilotNetSNN_66x200_gray_seed_42"   # ← nombre del run o None
# RUN_DIR_MANUAL = "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42"
# RUN_DIR_MANUAL = "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_best_as_gr050_lam020_ema090_crop_06122025_rate_model-PilotNetSNN_66x200_gray_seed_42"
# RUN_DIR_MANUAL = "continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42"
RUN_DIR_MANUAL = "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_best_as_gr050_lam020_ema090_rate_model-PilotNetSNN_66x200_gray_seed_42"
PRESET_MANUAL  = "std"  # preset con el que entrenaste ese run

# === 1) Elegir run: manual o desde best_for_sim.json =========================
if RUN_DIR_MANUAL is not None:
    # Ruta manual: usamos exactamente el nombre de carpeta bajo outputs/
    RUN_DIR_REL = RUN_DIR_MANUAL
    PRESET = PRESET_MANUAL or "std"
    print("[INFO] Usando RUN_DIR_MANUAL:", RUN_DIR_REL)
else:
    # Ruta original: se apoya en best_for_sim.json
    BEST_JSON = ROOT / "outputs" / "best_for_sim.json"
    if not BEST_JSON.exists():
        raise FileNotFoundError(
            f"No existe {BEST_JSON}. "
            f"Pon RUN_DIR_MANUAL en vez de usar best_for_sim.json, "
            f"o genera primero ese fichero desde el notebook de análisis."
        )

    best = json.loads(BEST_JSON.read_text(encoding="utf-8"))
    RUN_DIR_REL = best["run_dir"]
    PRESET = best.get("preset", "accurate")
    print("[INFO] Usando run de best_for_sim.json:", RUN_DIR_REL)

RUN_DIR_ABS = ROOT / "outputs" / RUN_DIR_REL
if not RUN_DIR_ABS.exists():
    raise FileNotFoundError(f"RUN_DIR_ABS no existe: {RUN_DIR_ABS}")

MODEL_NAME = "pilotnet_snn"   # el modelo que has usado en los runs CL
PORT = 4567

print("ROOT:", ROOT)
print("RUN_DIR_REL:", RUN_DIR_REL)
print("RUN_DIR_ABS:", RUN_DIR_ABS)
print("PRESET:", PRESET)

# === 2) Buscar checkpoint en task_2_circuito2 de ese run =====================
task2 = RUN_DIR_ABS / "task_2_circuito2"
if not task2.exists():
    raise FileNotFoundError(f"No existe carpeta task_2_circuito2 en {task2}")

ckpt_candidates = list(task2.glob("*.pt")) + list(task2.glob("*.pth"))
if not ckpt_candidates:
    raise FileNotFoundError(f"No se encontraron checkpoints (*.pt|*.pth) en {task2}")

def _pick_ckpt(files):
    best_like = [p for p in files if "best" in p.stem.lower()]
    if best_like:
        return max(best_like, key=lambda p: p.stat().st_mtime)
    return max(files, key=lambda p: p.stat().st_mtime)

CKPT = _pick_ckpt(ckpt_candidates)
print("CKPT seleccionado:", CKPT)

# === 3) Leer el preset para recuperar parámetros de datos/modelo =============
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)
DATA, MODEL = CFG["data"], CFG["model"]

ENCODER = DATA["encoder"]
T        = int(DATA["T"])
GAIN     = float(DATA["gain"])
W        = int(MODEL["img_w"])
H        = int(MODEL["img_h"])
TO_GRAY  = bool(MODEL["to_gray"])

print(f"[preset={PRESET}] model={MODEL_NAME} {W}x{H} gray={TO_GRAY} | enc={ENCODER} T={T} gain={GAIN}")


[INFO] Usando RUN_DIR_MANUAL: continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42
ROOT: /home/cesar/proyectos/TFM_SNN
RUN_DIR_REL: continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42
RUN_DIR_ABS: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42
PRESET: std
CKPT seleccionado: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42/task_2_circuito2/best.pth
[preset=std] model=pilotnet_snn 200x66 gray=False | enc=rate T=18 gain=0.7


In [3]:
import torch

# Construye el modelo y carga el checkpoint para validar compatibilidad
tfm = ImageTransform(W, H, to_gray=TO_GRAY, crop_top=None)
model = build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)

state = torch.load(CKPT, map_location="cpu")

if isinstance(state, dict):
    if "state_dict" in state:
        sd = state["state_dict"]
    elif "model_state_dict" in state:
        sd = state["model_state_dict"]
    else:
        sd = state
else:
    sd = state

model.load_state_dict(sd)
model.eval()

print("OK: modelo cargado y listo para inferencia con ese checkpoint.")


OK: modelo cargado y listo para inferencia con ese checkpoint.


In [4]:
# Construye el comando para sim_drive.py.
# Ojo: no forzamos encoder/T/gain/img_w/img_h por CLI,
# se cogen del preset. Así evitas inconsistencias.

cmd = f"""python tools/sim_drive.py \
  --ckpt {shlex.quote(str(CKPT))} \
  --model-name {MODEL_NAME} \
  --preset {PRESET} \
  --port {PORT} \
  --target-speed 12""".strip()

print("Comando:\n", cmd)

# Lanza el servidor como proceso hijo
proc = subprocess.Popen(cmd, shell=True, cwd=ROOT)
print("Servidor lanzado. PID =", proc.pid)


Comando:
 python tools/sim_drive.py   --ckpt /home/cesar/proyectos/TFM_SNN/outputs/continual_std_sa-snn_best_sa_k013_vt120_crop_06122025_rate_model-PilotNetSNN_66x200_rgb_seed_42/task_2_circuito2/best.pth   --model-name pilotnet_snn   --preset std   --port 4567   --target-speed 12
Servidor lanzado. PID = 369934


Ahora abre el **Udacity Simulator** y entra en **Autonomous Mode**.
Deberías ver cómo el coche comienza a moverse. Vuelve aquí para ver logs o parar el servidor.


In [5]:
import os, signal, time

time.sleep(2.0)  # pequeña espera para que arranque

if proc and (proc.poll() is None):
    print(f"Servidor VIVO (PID={proc.pid})")
else:
    print("Servidor NO está vivo (probablemente ha petado; mira la consola).")

# Para detenerlo manualmente:
# if proc and (proc.poll() is None):
#     os.kill(proc.pid, signal.SIGTERM)  # en Windows: proc.terminate()
#     time.sleep(1.0)
#     print("Servidor detenido.")


Servidor VIVO (PID=369934)


[sim] dispositivo=cuda | AMP=ON
[sim] modelo=pilotnet_snn | 200x66 gray=False | enc=rate T=18 gain=0.7
[sim] escuchando en http://0.0.0.0:4567
